In [ ]:
import numpy as np
import torch
import torchwordemb

from matplotlib import pyplot as plt
from torch import optim


from models_code.mnist import MonteCarloDropout
from models_code.experiments import test_eval
from models_code.experiments import test_eval_variational
from models_code.experiments import correlation_test_error_uncertainty
from models_code.experiments import non_distribution
from models_code.experiments import correlation_test_error_uncertainty_variational
from models_code.experiments import softmax2d
from models_code.sentiment import generator_out_of_matrix
from models_code.sentiment import Movie
from models_code.sentiment import MCMovie
from models_code.sentiment import ISMovie
from models_code.bayesbybackprop import BBPMovie
from models_code.sentiment import perform_training_sentiment
from models_code.utilities import load_model
from models_code.utilities import create_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

from utilities.metric import predictive_entropy
from utilities.metric import entropy

In [ ]:
train_matrix = np.load(open('./movies/train_e.npy', "rb"))
test_matrix = np.load(open('./movies/test_e.npy', "rb"))
labels = np.zeros(25000, dtype=np.int)
labels[:12500] = 1

# Standard network

In [ ]:
standard, optimizer, loss_function = create_model(
    Movie,
    loss_function=torch.nn.BCEWithLogitsLoss,
    optimizer=lambda x: optim.RMSprop(x, lr=0.0005)
)

perform_training_sentiment(
    60,
    standard,
    train_matrix,
    test_matrix,
    labels,
    optimizer,
    loss_function,
    100,
    'models/sentiment/standard.torch',
    100
)

# standard = load_model(Movie, 'models/sentiment/standard.torch')

### Experiment 1

In [ ]:
test_preds, test_labels, test_probs = test_eval(
    standard,
    generator_out_of_matrix(test_matrix, labels, 100, shuffle=False),
    sentiment=True
)
print(accuracy_score(labels, test_probs > 0))

print(log_loss(labels, test_preds, eps=1e-7))

### Experiment 2

In [ ]:


roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty(
    lambda x: 1-np.abs(x-0.5),
    test_preds,
    test_labels,
    num_classes=2
)
print(roc)
print(ac)

### Experiment 3

In [ ]:
crd_matrix = np.load(open('./nonmovies/crd_e.npy', "rb"))
rp_matrix = np.load(open('./nonmovies/rp_e.npy', "rb"))
rp_preds, rp_labels, rp_probs = test_eval(
    standard,
    generator_out_of_matrix(rp_matrix, np.zeros(rp_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
rp_labels = np.zeros(2000, dtype=np.int)
rp_labels[:1000] = 1
print(accuracy_score(rp_labels, rp_preds > 0.5))
print(log_loss(rp_labels, rp_preds, eps=1e-7))

In [ ]:
crd_preds, crd_labels, crd_probs = test_eval(
    standard,
    generator_out_of_matrix(crd_matrix, np.zeros(crd_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    -np.abs(0.5-test_preds),
    -np.abs(0.5-crd_preds),
    25314,
    25000
)
print(roc)
print(ac)

In [ ]:
reuters_matrix = np.load(open('./nonmovies/reuters_e.npy', "rb"))
news_matrix = np.load(open('./nonmovies/news_e.npy', "rb"))
reuters_preds, reuters_labels, reuters_probs = test_eval(
    standard,
    generator_out_of_matrix(reuters_matrix, np.zeros(reuters_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    -np.abs(0.5-test_preds),
    -np.abs(0.5-reuters_preds),
    46578,
    25000
)
print(roc)
print(ac)

In [ ]:
news_preds, news_labels, news_probs = test_eval(
    standard,
    generator_out_of_matrix(news_matrix, np.zeros(news_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    -np.abs(0.5-test_preds),
    -np.abs(0.5-news_preds),
    43828,
    25000
)
print(roc)
print(ac)

# MC Dropout

In [ ]:
# mc, optimizer, loss_function = create_model(
#     MCMovie,
#     loss_function=torch.nn.BCEWithLogitsLoss,
#     optimizer=lambda x: optim.RMSprop(x, lr=0.0005)
# )

# perform_training_sentiment(
#     60,
#     mc,
#     train_matrix,
#     test_matrix,
#     labels,
#     optimizer,
#     loss_function,
#     100,
#     'models/sentiment/mc2.torch',
#     100
# )

mc = load_model(MCMovie, 'models/sentiment/mc2.torch')

In [ ]:
mc.dropout = MonteCarloDropout(0.5)

In [ ]:
test_labels, test_preds = test_eval_variational(
    mc,
    generator_out_of_matrix(test_matrix, labels, 100, shuffle=False),
    50,
    sentiment=True
)

In [ ]:
accuracy_score(test_labels, np.mean(test_preds, axis=0) > 0.5)

In [ ]:
log_loss(test_labels, np.mean(test_preds, axis=0), eps=1e-7)

### Second experiment

In [ ]:
roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty_variational(
    predictive_entropy,
    test_preds,
    test_labels
)

In [ ]:
roc

In [ ]:
ac

### Third experiment

In [ ]:
crd_matrix = np.load(open('./nonmovies/crd_e.npy', "rb"))
rp_matrix = np.load(open('./nonmovies/rp_e.npy', "rb"))
rp_labels, rp_probs = test_eval_variational(
    mc,
    generator_out_of_matrix(rp_matrix, np.zeros(rp_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    50,
    sentiment=True
)
rp_labels = np.zeros(2000, dtype=np.int)
rp_labels[:1000] = 1
accuracy_score(rp_labels, np.mean(rp_probs,axis=0) > 0.5)

In [ ]:
log_loss(rp_labels, np.mean(rp_probs,axis=0), eps=1e-7)

In [ ]:
crd_labels, crd_preds = test_eval_variational(
    mc,
    generator_out_of_matrix(crd_matrix, np.zeros(crd_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    50,
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-test_preds[:,:,0], test_preds[:,:,0]], axis=2)).reshape(25000,1),    
    predictive_entropy(np.stack([1-crd_preds[:,:,0], crd_preds[:,:,0]], axis=2)).reshape(314,1),
    25314,
    25000
)
roc

In [ ]:
ac

In [ ]:
reuters_matrix = np.load(open('./nonmovies/reuters_e.npy', "rb"))
news_matrix = np.load(open('./nonmovies/news_e.npy', "rb"))
reuters_labels, reuters_preds = test_eval_variational(
    mc,
    generator_out_of_matrix(reuters_matrix, np.zeros(reuters_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    50,
    sentiment=True
)


roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-test_preds[:,:,0], test_preds[:,:,0]], axis=2)).reshape(25000,1),
    predictive_entropy(np.stack([1-reuters_preds[:,:,0], reuters_preds[:,:,0]], axis=2)).reshape(21578,1),
    46578,
    25000
)
roc

In [ ]:
ac

In [ ]:
news_labels, news_preds = test_eval_variational(
    mc,
    generator_out_of_matrix(news_matrix, np.zeros(news_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    50,
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-test_preds[:,:,0], test_preds[:,:,0]], axis=2)).reshape(25000,1),
    predictive_entropy(np.stack([1-news_preds[:,:,0], news_preds[:,:,0]], axis=2)).reshape(18828,1),
    43828,
    25000
)
roc

In [ ]:
ac

### Inhibited softmax

In [ ]:
is_, optimizer, loss_function = create_model(
    ISMovie,
    loss_function=torch.nn.CrossEntropyLoss,
    optimizer=lambda x: optim.RMSprop(x, lr=0.0005)
)

# def is_loss(model):
    
#     return (
#         lambda pred,y: loss_function(pred,y)
#         + 0.01 * (model.dense.weight.data ** 2).sum()
#         + 0.000001 * pred.sum()
#     )

# perform_training_sentiment(
#     60,
#     is_,
#     train_matrix,
#     test_matrix,
#     labels,
#     optimizer,
#     is_loss(is_),
#     100,
#     'models/sentiment/is.torch',
#     100,
#     bce=False
# )


is_ = load_model(ISMovie, 'models/sentiment/is.torch')

In [ ]:
test_preds, test_labels, test_probs = test_eval(
    is_,
    generator_out_of_matrix(test_matrix, labels, 100, shuffle=False),
    is_sentiment=True
)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
accuracy_score(test_labels, np.argmax(test_probs, axis=1))

In [ ]:
log_loss(test_labels, softmax2d(test_probs[:,:2])[:,1])

### Second experiment

In [ ]:


roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty(
    lambda x: softmax2d(x)[:,2],
    test_probs,
    test_labels,
    num_classes=2
)
# roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty(
#     lambda x: entropy(softmax2d(x)),
#     test_probs,
#     test_labels,
#     num_classes=2
# )

roc

In [ ]:
ac

### Third experiment

In [ ]:
crd_matrix = np.load(open('./nonmovies/crd_e.npy', "rb"))
rp_matrix = np.load(open('./nonmovies/rp_e.npy', "rb"))
rp_preds, rp_labels, rp_probs = test_eval(
    is_,
    generator_out_of_matrix(rp_matrix, np.zeros(rp_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
rp_labels = np.zeros(2000, dtype=np.int)
rp_labels[:1000] = 1
accuracy_score(rp_labels, rp_preds > 0.5)

In [ ]:
log_loss(rp_labels, softmax2d(rp_probs[:,:2])[:,1])

In [ ]:
crd_preds, crd_labels, crd_probs = test_eval(
    is_,
    generator_out_of_matrix(crd_matrix, np.zeros(crd_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    softmax2d(test_probs)[:,2],
    softmax2d(crd_probs)[:,2],
    25314,
    25000
)
# roc, ac, fpr, tpr, pr, re = non_distribution(
#     test_probs,
#     entropy(softmax2d(test_probs)),
#     entropy(softmax2d(crd_probs)),
#     25314,
#     25000
# )
roc

In [ ]:
ac

In [ ]:
reuters_matrix = np.load(open('./nonmovies/reuters_e.npy', "rb"))
news_matrix = np.load(open('./nonmovies/news_e.npy', "rb"))
reuters_preds, reuters_labels, reuters_probs = test_eval(
    is_,
    generator_out_of_matrix(reuters_matrix, np.zeros(reuters_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    softmax2d(test_probs)[:,2],
    softmax2d(reuters_probs)[:,2],
    46578,
    25000
)

# roc, ac, fpr, tpr, pr, re = non_distribution(
#     test_probs,
#     entropy(softmax2d(test_probs)),
#     entropy(softmax2d(reuters_probs)),
#     46578,
#     25000
# )
roc

In [ ]:
ac

In [ ]:
news_preds, news_labels, news_probs = test_eval(
    is_,
    generator_out_of_matrix(news_matrix, np.zeros(news_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    softmax2d(test_probs)[:,2],
    softmax2d(news_probs)[:,2],
    43828,
    25000
)


# roc, ac, fpr, tpr, pr, re = non_distribution(
#     test_probs,
#     entropy(softmax2d(test_probs)),
#     entropy(softmax2d(news_probs)),
#     43828,
#     25000
# )
roc

In [ ]:
ac

### Deep Ensembles

In [ ]:
for i in range(5):

    standard, optimizer, loss_function = create_model(
        Movie,
        loss_function=torch.nn.BCEWithLogitsLoss,
        optimizer=lambda x: optim.RMSprop(x, lr=0.0005)
    )

    perform_training_sentiment(
        60,
        standard,
        train_matrix,
        test_matrix,
        labels,
        optimizer,
        loss_function,
        100,
        'models/sentiment/de{}.torch'.format(i),
        100
    )


# models = [load_model(Movie, 'models/sentiment/de{}.torch'.format(i)) for i in range(5)]

In [ ]:
#test_preds, test_labels, test_probs = test_eval(
test_results = [test_eval(
    model,
    generator_out_of_matrix(test_matrix, labels, 100, shuffle=False),
    sentiment=True
) for model in models]

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
test_probs = np.mean(np.stack([sigmoid(m[2])for m in test_results]), axis=0)
accuracy_score(
    labels,
    test_probs > 0.5
)

In [ ]:
log_loss(
    labels,
    test_probs,
    eps=1e-7
)

### Second experiment

In [ ]:
roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty_variational(
    predictive_entropy,
    np.stack([sigmoid(m[2])for m in test_results]),
    labels
)

In [ ]:
roc

In [ ]:
ac


### Third experiment

In [ ]:
crd_matrix = np.load(open('./nonmovies/crd_e.npy', "rb"))
rp_matrix = np.load(open('./nonmovies/rp_e.npy', "rb"))

rp_results = [test_eval(
    model,
    generator_out_of_matrix(rp_matrix, np.zeros(rp_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    is_sentiment=True
) for model in models]

rp_probs = np.stack([sigmoid(rp[2]) for rp in rp_results])
rp_labels = np.zeros(2000, dtype=np.int)
rp_labels[:1000] = 1
accuracy_score(rp_labels, np.mean(rp_probs,axis=0) > 0.5)

In [ ]:
log_loss(rp_labels, np.mean(sigmoid(rp_probs),axis=0), 1e-7)

In [ ]:
dual_preds = np.stack([sigmoid(m[2])for m in test_results])[:,:,0].T

In [ ]:
# crd_preds, crd_labels, crd_probs = test_eval(
crd_results = [test_eval(
    model,
    generator_out_of_matrix(crd_matrix, np.zeros(crd_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
) for model in models]

crd_probs = np.stack([sigmoid(crd[2]) for crd in crd_results])

roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-dual_preds.T, dual_preds.T], axis=2)).reshape(25000,1),
    predictive_entropy(np.stack([1-crd_probs[:,:,0], crd_probs[:,:,0]], axis=2)).reshape(314,1),
    25314,
    25000
)
roc

In [ ]:
ac

In [ ]:
reuters_matrix = np.load(open('./nonmovies/reuters_e.npy', "rb"))
news_matrix = np.load(open('./nonmovies/news_e.npy', "rb"))

reuters_results = [test_eval(
    model,
    generator_out_of_matrix(reuters_matrix, np.zeros(reuters_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
) for model in models]

reuters_probs = np.stack([sigmoid(reuters[2]) for reuters in reuters_results])



roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-dual_preds.T, dual_preds.T], axis=2)).reshape(25000,1),
    predictive_entropy(np.stack([1-reuters_probs[:,:,0], reuters_probs[:,:,0]], axis=2)).reshape(21578,1),

    46578,
    25000
)
roc

In [ ]:
ac

In [ ]:
from utilities.metric import expected_entropy

In [ ]:

news_results = [test_eval(
    model,
    generator_out_of_matrix(news_matrix, np.zeros(news_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
) for model in models]

news_probs = np.stack([sigmoid(news[2]) for news in news_results])


roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-dual_preds.T, dual_preds.T], axis=2)).reshape(25000,1),
    predictive_entropy(np.stack([1-news_probs[:,:,0], news_probs[:,:,0]], axis=2)).reshape(18828,1),

    43828,
    25000
)
roc

In [ ]:
ac

### Bayes by backprop

In [ ]:
bbp, optimizer, loss_function = create_model(
    BBPMovie,
    loss_function=torch.nn.BCEWithLogitsLoss,
    optimizer=lambda x: optim.RMSprop(x, lr=0.0005)
)

perform_training_sentiment(
    60,
    bbp,
    train_matrix,
    test_matrix,
    labels,
    optimizer,
    loss_function,
    100,
    'models/sentiment/bbp.torch',
    100
)

# standard = load_model(Movie, 'models/sentiment/standard.torch')

In [ ]:
test_labels, test_preds = test_eval_variational(
    bbp,
    generator_out_of_matrix(test_matrix, labels, 100, shuffle=False),
    10,
    sentiment=True
)

In [ ]:
accuracy_score(test_labels, np.mean(test_preds, axis=0) > 0.5)

In [ ]:
log_loss(test_labels, np.mean(test_preds, axis=0), eps=1e-7)

### Second experiment

In [ ]:
roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty_variational(
    predictive_entropy,
    test_preds,
    test_labels
)

In [ ]:
roc

In [ ]:
ac

### Third experiment

In [ ]:
crd_matrix = np.load(open('./nonmovies/crd_e.npy', "rb"))
rp_matrix = np.load(open('./nonmovies/rp_e.npy', "rb"))
rp_labels, rp_probs = test_eval_variational(
    bbp,
    generator_out_of_matrix(rp_matrix, np.zeros(rp_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    10,
    sentiment=True
)
rp_labels = np.zeros(2000, dtype=np.int)
rp_labels[:1000] = 1
accuracy_score(rp_labels, np.mean(rp_probs,axis=0) > 0.5)

In [ ]:
log_loss(rp_labels, np.mean(rp_probs,axis=0), eps=1e-7)

In [ ]:
crd_labels, crd_preds = test_eval_variational(
    bbp,
    generator_out_of_matrix(crd_matrix, np.zeros(crd_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    10,
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-test_preds[:,:,0], test_preds[:,:,0]], axis=2)).reshape(25000,1),    
    predictive_entropy(np.stack([1-crd_preds[:,:,0], crd_preds[:,:,0]], axis=2)).reshape(314,1),
    25314,
    25000
)
roc

In [ ]:
ac

In [ ]:
reuters_matrix = np.load(open('./nonmovies/reuters_e.npy', "rb"))
news_matrix = np.load(open('./nonmovies/news_e.npy', "rb"))
reuters_labels, reuters_preds = test_eval_variational(
    bbp,
    generator_out_of_matrix(reuters_matrix, np.zeros(reuters_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    10,
    sentiment=True
)


roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-test_preds[:,:,0], test_preds[:,:,0]], axis=2)).reshape(25000,1),
    predictive_entropy(np.stack([1-reuters_preds[:,:,0], reuters_preds[:,:,0]], axis=2)).reshape(21578,1),
    46578,
    25000
)
roc

In [ ]:
ac

In [ ]:
news_labels, news_preds = test_eval_variational(
    bbp,
    generator_out_of_matrix(news_matrix, np.zeros(news_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    10,
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    None,
    predictive_entropy(np.stack([1-test_preds[:,:,0], test_preds[:,:,0]], axis=2)).reshape(25000,1),
    predictive_entropy(np.stack([1-news_preds[:,:,0], news_preds[:,:,0]], axis=2)).reshape(18828,1),
    43828,
    25000
)
roc

In [ ]:
ac

# Standard modified

In [ ]:
standard, optimizer, loss_function = create_model(
    Movie,
    loss_function=torch.nn.BCEWithLogitsLoss,
    optimizer=lambda x: optim.RMSprop(x, lr=0.0005)
)

def is_loss(model):
    
    return (
        lambda pred,y: loss_function(pred,y)
        + 0.01 * (model.dense.weight.data ** 2).sum()
        + 0.000001 * pred.sum()
    )

# perform_training_sentiment(
#     60,
#     standard,
#     train_matrix,
#     test_matrix,
#     labels,
#     optimizer,
#     is_loss(standard),
#     100,
#     'models/sentiment/standard_mod.torch',
#     100
# )

standard = load_model(Movie, 'models/sentiment/standard.torch')

### experiment 1

In [ ]:
test_preds, test_labels, test_probs = test_eval(
    standard,
    generator_out_of_matrix(test_matrix, labels, 100, shuffle=False),
    sentiment=True
)
print(accuracy_score(labels, test_probs > 0))

print(log_loss(labels, test_preds, eps=1e-7))

### experiment 2

In [ ]:


roc, ac, fpr, tpr, pr, re = correlation_test_error_uncertainty(
    lambda x: 1-np.abs(x-0.5),
    test_preds,
    test_labels,
    num_classes=2
)
print(roc)
print(ac)

### experiment 3

In [ ]:

crd_matrix = np.load(open('./nonmovies/crd_e.npy', "rb"))
rp_matrix = np.load(open('./nonmovies/rp_e.npy', "rb"))
rp_preds, rp_labels, rp_probs = test_eval(
    standard,
    generator_out_of_matrix(rp_matrix, np.zeros(rp_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
rp_labels = np.zeros(2000, dtype=np.int)
rp_labels[:1000] = 1
print(accuracy_score(rp_labels, rp_preds > 0.5))
print(log_loss(rp_labels, rp_preds, eps=1e-7))

In [ ]:
crd_preds, crd_labels, crd_probs = test_eval(
    standard,
    generator_out_of_matrix(crd_matrix, np.zeros(crd_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    -np.abs(0.5-test_preds),
    -np.abs(0.5-crd_preds),
    25314,
    25000
)
print(roc)
print(ac)

In [ ]:
reuters_matrix = np.load(open('./nonmovies/reuters_e.npy', "rb"))
news_matrix = np.load(open('./nonmovies/news_e.npy', "rb"))
reuters_preds, reuters_labels, reuters_probs = test_eval(
    standard,
    generator_out_of_matrix(reuters_matrix, np.zeros(reuters_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    -np.abs(0.5-test_preds),
    -np.abs(0.5-reuters_preds),
    46578,
    25000
)
print(roc)
print(ac)

In [ ]:
news_preds, news_labels, news_probs = test_eval(
    standard,
    generator_out_of_matrix(news_matrix, np.zeros(news_matrix.shape[0], dtype=np.int), 100, shuffle=False),
    sentiment=True
)
roc, ac, fpr, tpr, pr, re = non_distribution(
    test_probs,
    -np.abs(0.5-test_preds),
    -np.abs(0.5-news_preds),
    43828,
    25000
)
print(roc)
print(ac)